In [1]:
import pandas as pd
from datetime import timedelta,datetime
import time
import openpyxl

class LuXian():
  # 对应于excel表格的路线表
    def __init__ (self,lx:pd.DataFrame):
        # 初始化格式
        self.lx = lx.astype({
        '站点编号':'int'
        })
        
    def get_shuangxiang(self) -> pd.DataFrame:
        # 1. 输入参数为单向的路线编号及距离信息
        # 2. 输出参数为双向的路线编号及距离信息
        
        #正向路线标识
            lx1 = self.lx.assign(**{
                '方向':'往'
            })
        #根据正向路线生成逆向路线
            lx2 = lx1.copy().sort_values('站点编号',ascending=False)
            lx2 = lx2.assign(**{
            '距离':lx2['距离'].shift(1).fillna(0),
            '方向':'返'
            }).reset_index(drop=True)
        
            return  pd.concat([lx1,lx2]).reset_index(drop=True)

class PaiBan():
    # 对应于excel表格的排版表
    def __init__(self,pb:pd.DataFrame) -> pd.DataFrame:

        # 初始化排版时，转换日期格式
        
        pb['发车时间']= pb.apply(lambda t:(datetime.combine(pd.Timestamp.today().normalize(),t['发车时间'])-pd.Timestamp.today().normalize()),axis=1)
        self.paiban = pb

    def get_paibai(self):
        return self.paiban

class CheLiang():
    # 对应于excel表格的车辆表
    def __init__ (self,cl:pd.DataFrame) -> pd.DataFrame:
        cl.astype({
            '座位':'int'
        })
        self.cl = cl
        
    

    def get_cheliang(self):
        return self.cl
class GouPiao():
    def __init__(self):
        self.goupiao = pd.DataFrame()
        pass
       
        

    def _maipiao(self,checi:str,up:int ,down:int,sch:pd.DataFrame):
        #获取剩余票量
        mask = (sch['车次']== checi) & (sch['站点编号'] >= up) &(sch['站点编号']<= down) & (sch['可售座位']>0)
        ss = sch[mask]
        if ss.shape[0]>0 :
            sch.loc[mask,'可售座位']-=1
            if self.goupiao.empty == False:
                # print("前面",self.goupiao)
                self.goupiao = pd.concat([self.goupiao,pd.DataFrame([{'操作时间':datetime.now(),'车次':checi,'入站':up,'出站':down}])],ignore_index=
                                    True)
                # print('后面',self.goupiao)
            else:
                self.goupiao = pd.DataFrame([{'操作时间':datetime.now(),'车次':checi,'入站':up,'出站':down}])

        else:
            print ("选定路程票已卖光")

        return sch
    def get_piaocang(self):
        return self.goupiao
        
        

class WorkStation():
    
        
    def __init__(self,luxian:pd.DataFrame,paiban:pd.DataFrame,car:pd.DataFrame) -> pd.DataFrame:
        sch1 = pd.merge(paiban,luxian,on='方向',how='inner').merge(car,on='车辆编号',how='left')
        sch1 = sch1.assign(**{
            '耗时':sch1.apply(lambda row:timedelta( hours=row['距离']/row['速度'] ) if row['速度']!=0 else np.na,axis=1)
        })
        sch1['累计耗时'] = sch1.groupby(['车辆编号','发车时间','方向'])['耗时'].cumsum()
        sch1 = sch1.assign(**{
            '发车时间':sch1.apply(lambda row:row['发车时间']+row['累计耗时'] ,axis=1)
        })
        sch1['可售座位']= sch1['座位']
        self.sch = sch1
        self.piaocang = None
        self.__distance()#调用计算距离
        
    def get_schedule(self):
        return self.sch
        
    def display_schedule(self):
        lx1 = self.sch
        lx1['发车时间'] = lx1.apply(lambda row:(pd.Timestamp.today().normalize()+row['发车时间']).time(),axis=1)
        lx1['耗时'] = lx1.apply(lambda row:(pd.Timestamp.today().normalize()+row['耗时']).time(),axis=1)
        pd.set_option('display.max_colwidth', 500)
        
        print("\n列车时间表为：\n",lx1[['车次','站点名称','发车时间','耗时','座位','总里程','可售座位']])

    def display_sold(self):
        if self.piaocang is not None:
            lx1 = self.piaocang
            print("\n购票记录为:\n",lx1.get_piaocang())

        else:
            print("\n暂无购票记录\n")

    def savetoexcel(self,path:str):
        with pd.ExcelWriter(path) as writer:
            self.sch.to_excel(writer,index=False,sheet_name='列车时间表',engine='openpyxl')
            self.piaocang.get_piaocang().to_excel(writer,index=False,sheet_name='购票记录',engine='openpyxl')
        
    def __distance(self):
        # 获取车辆移动的距离，W=FS，知道距离就知道做了多少功，根据车辆的转换效率很容易算出消耗的电、煤、油的数量和价格
        sch1 = self.sch
        sch1['总里程'] = sch1.groupby(['车辆编号'])['距离'].cumsum()
        self.sch = sch1

    def maipiao(self,checi:str,up:int ,down:int):
        self.sch = getattr(self.piaocang,'_maipiao')(checi,up,down,self.sch)
        

    def set_goupiao(self,gp:GouPiao):
        self.piaocang = gp
        


#读取数据
ex =  pd.read_excel("/Users/henry/Downloads/pandas/交通系统-输入.xls",sheet_name=None)
lx = LuXian(ex['路线']).get_shuangxiang()
pb = PaiBan(ex['排班表']).get_paibai()
cl =CheLiang(ex['车辆']).get_cheliang()
wk =WorkStation(lx,pb,cl)
wk.set_goupiao(GouPiao())
for i in range(0,12):
    print(i)
    time.sleep(1)
    wk.maipiao('G001',2,5)
# wk.maipiao('G001',3,5)
wk.display_schedule()
wk.display_sold()
wk.savetoexcel('/Users/henry/Downloads/pandas/交通系统-输出.xlsx')



0
1
2
3
4
5
6
7
8
9
10
选定路程票已卖光
11
选定路程票已卖光

列车时间表为：
       车次  站点名称      发车时间        耗时  座位    总里程  可售座位
0   G001    莘庄  08:00:00  00:00:00  10    0.0    10
1   G001   外环路  08:33:00  00:33:00  10   22.0     0
2   G001   莲花路  09:13:30  00:40:30  10   49.0     0
3   G001  锦江乐园  10:00:00  00:46:30  10   80.0     0
4   G001    南站  10:37:30  00:37:30  10  105.0     0
5   G001   漕宝路  11:22:30  00:45:00  10  135.0    10
6   G001   体育馆  11:54:00  00:31:30  10  156.0    10
7   G001   徐家汇  12:39:00  00:45:00  10  186.0    10
8   G001   衡山路  13:25:30  00:46:30  10  217.0    10
9   G001   常熟路  14:01:30  00:36:00  10  241.0    10
10  G001   陕西路  14:37:30  00:36:00  10  265.0    10
11  G001  人民广场  15:22:30  00:45:00  10  295.0    10
12  G002  人民广场  12:00:00  00:00:00  10  295.0    10
13  G002   陕西路  12:45:00  00:45:00  10  325.0    10
14  G002   常熟路  13:21:00  00:36:00  10  349.0    10
15  G002   衡山路  13:57:00  00:36:00  10  373.0    10
16  G002   徐家汇  14:43:30  00:46:30  10  404.0    10
17  G002  